<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストデータの前処理

* テキストデータは、長い長い文字列。
* 長い長い文字列のままでは、普通は分析できない。
* 今回は、基本的な前処理について学ぶ。
 * 英語と日本語の両方。

## str型のメソッドによる前処理

* 例えば、大文字と小文字の間の変換などが実行できる。

 * 問：元のテキストにあった大文字と小文字の区別を無くしてしまうことのメリットとデメリットは何か？

In [ ]:
text = "The quick brown fox jumped over The Big Dog"
text

In [ ]:
text.lower()

In [ ]:
text.upper()

In [ ]:
text.lower().split()

## NLTK

* https://www.nltk.org/

* Pythonで自然言語処理を行うためのライブラリ
  * 2001年スタートらしい。

* WordNetも使える（ここでは説明しない）
 * https://www.nltk.org/howto/wordnet.html
 * WordNetについては『IT Text 自然言語処理の基礎』3.2.2(a)を参照。

In [ ]:
import nltk

### NLTKによるトークン化

* 文に分ける、単語に分ける、など、長い文字列としての言語データをより小さな単位へと分割することを、一般にtokenizationと言う。

* Pythonの文字列は、複数行にわたっていても、丸括弧でくくれば一つの長い文字列になる。
 * ただし、最後の行を除いて、末尾に空白を入れておくのを忘れないように。

In [ ]:
text = (
    "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism. "
    'Sometimes called "the father of modern linguistics", Chomsky is also a major figure in analytic philosophy and one of the founders of the field of cognitive science. '
    "He is a laureate professor of linguistics at the University of Arizona and an institute professor emeritus at the Massachusetts Institute of Technology (MIT)."
)
text

* 文へtokenize
 * `punkt`というパッケージが必要なのでダウンロードしておく。

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.sent_tokenize(text)

* 単語へtokenize

In [ ]:
print(nltk.word_tokenize(text))

## spaCy

* spaCyも、Pythonの有名な自然言語処理ライブラリ。2015年スタートらしい。

* https://spacy.io/

In [ ]:
import spacy

### spaCyの「モデル」
* 各言語ごとにモデルが用意されている。
 * https://spacy.io/models
* モデルにはいくつか種類があり、規模が異なる。
 * 大きなモデルでは、かなり複雑な処理ができる。

### Tokenization

* モデルをロードする。

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(text)

* 文へtokenize

In [ ]:
[sent.text for sent in doc.sents]

* 単語へtokenize

In [ ]:
print([token.text for token in doc])

* 品詞

In [ ]:
print([token.pos_ for token in doc])

## 例題: HTML文書

* __`<p>`__や__`<a>`__や__`<div>`__など、頻繁に使うHTMLタグは頭に入れておいてください。

* なぜなら、ある程度HTMLタグが読めてはじめて、スクレイピングのコードを書くための、HTMLソースの下調べができるからです。
 * 自前でWeb上から分析対象のテキストデータを取得するときは、ダウンロードしようとするWebページのHTMLの構造を自分の目で確認する。

* 注意: スクレイピングを禁止しているWebサイトもあるので注意しましょう。

### HTML文書のダウンロード
* いくつか方法はあるが、ここではrequestsモジュールを使う。

In [ ]:
import requests

data = requests.get('http://www.gutenberg.org/cache/epub/8001/pg8001.html')
content = data.text
print(content[4000:6000])

### Beautiful Soup

* HTML文書の構造を解析するためによく使われるライブラリ。

* 参考資料：「Beautiful Soup 4によるスクレイピングの基礎」

 * https://www.atmarkit.co.jp/ait/articles/1910/18/news015.html

* 注意：スクレイピングを禁止しているWebサイトもあるので注意しよう。

* HTMLタグを取り除くコードの例
 * 改行記号をまとめるために`re`（正規表現）モジュールを使っている。

In [ ]:
import re
from bs4 import BeautifulSoup

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  # 下の正規表現の意味を説明してみよう。
  return re.sub(r'[\r|\n|\r\n]+', '\n', soup.get_text())

clean_content = strip_html_tags(content)
print(clean_content[3000:5000])

# 演習
* 上の`clean_content`を単語に分割し、各単語の出現頻度を求め、出現頻度の高い順に上位100の単語を、出現頻度とともに表示しよう。

## 様々な前処理

### 特殊文字、数字、記号の除去

* 問：テキストデータの前処理において特殊文字、数字、記号などを除去することのメリットとデメリットは何か？

In [ ]:
import re

def remove_special_characters(text, remove_digits=False):
  pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
  text = re.sub(pattern, '', text)
  return text

In [ ]:
s = "Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂"
s

In [ ]:
remove_special_characters(s, remove_digits=True)

In [ ]:
remove_special_characters(s)

### Lemmatization

* 動詞や形容詞は原型に、名詞は単数形に、等と、単語の元々の形に直すこと。

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
  return [word.lemma_ for word in nlp(text)]

In [ ]:
lemmatize('lying')

In [ ]:
lemmatize('goes')

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."
print(lemmatize(text))

### ストップワード

* ストップワードとは、非常に頻繁に使われるため、言語データの分析にあまり役に立ちそうにない単語のこと。
* これこそが英語のストップワードだ！と言えるような決定的なストップワードのリストがあるわけではない。
 * 主要なNLPライブラリでは、あらかじめ用意されたストップワードのリストを使うことができる。
 * しかし、分析したいテキストデータに合わせて、ストップワードのリストをカスタマイズすることも、よくある。
* **ストップワードの除去は最近あまり行われなくなっている。**
 * 深層学習言語モデルでは、元のままのテキストをサブワードへ分割するため。
 * 深層学習言語モデルを使うと、むしろ、ストップワードの役割を尊重しつつ、テキストを分析できる。
 * ストップワードの除去は、BoWでテキストを分析するときには、今でも実施する。

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

print(STOP_WORDS)
print(len(STOP_WORDS))

In [ ]:
def remove_stopwords(text):
  stopwords = spacy.lang.en.stop_words.STOP_WORDS
  return [token.text for token in nlp(text) if token.text not in stopwords]

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."
print(remove_stopwords(text))

## 現代的なtokenization
* 図表は下記のブログ記事より。
 * https://ai.googleblog.com/2020/09/advancing-nlp-with-efficient-projection.html

![Segmentation.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/Segmentation.png)
![inherent_task_complexity.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/image3.png)

* 今は、tokenizationと言えば、ほぼ、サブワードに分けることを意味する。

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."

In [ ]:
print(tokenizer.tokenize(text))

### Byte-Pair Encoding (BPE)
 * https://huggingface.co/docs/transformers/tokenizer_summary

## 日本語テキスト

* Google Colabではインストールの必要あり。

In [ ]:
!python -m spacy download ja_core_news_sm

In [ ]:
import spacy
from spacy.lang.ja.examples import sentences

In [ ]:
text = (
    "ノーム・チョムスキーは1928年12月7日、アメリカ合衆国ペンシルベニア州フィラデルフィアのイースト・オーク・レーン近郊で生まれた。"
    "父ウィリアム・チョムスキーは当時ロシア帝国支配下のウクライナで生まれたが、戦乱を避けて1913年にアメリカへ渡った。"
    "メリーランド州ボルチモアの搾取工場で働き、貯蓄してジョンズ・ホプキンス大学で学んだ甲斐もあり市のヘブライ人系小学校教師の職を得た。"
)


In [ ]:
nlp = spacy.load("ja_core_news_sm")
doc = nlp(text)
for token in doc:
  print((token.text, token.lemma_, token.pos_))

# 課題 2
* livedoor ニュースコーパスでの、単語の出現回数を調べよう。
 * https://www.rondhuit.com/download.html#news%20corpus
 * 活用変化する単語は、原型に戻してから、出現回数を数えよう。

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

In [ ]:
tar_fname = "ldcc-20140209.tar.gz"

* このデータセットの前処理については、下の記事を参照。
 * https://tech.fusic.co.jp/posts/2021-04-23-bert-multi-classification/

In [ ]:
# 余分な括弧の除去
def remove_brackets(text):
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", text))

# このデータセットのフォーマットに従ったファイルの読み込み
def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = remove_brackets(title.decode('utf-8'))
  return title[:-1]

In [ ]:
import tarfile

corpus = []

with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

In [ ]:
nlp = spacy.load("ja_core_news_sm")
print([token.lemma_ for token in nlp(corpus[0])])